### JSONL 文本分词处理
该脚本用于对 JSONL 文件中的文本进行分词处理，并输出分词结果到新的 JSONL 文件。

In [ ]:
import os
import json
import jieba
import re

### 加载停用词表
该函数从指定的停用词文件加载停用词列表。

In [ ]:
def load_stopwords(stopword_path):
    """
    从停用词文件加载停用词列表
    """
    stopwords = set()
    with open(stopword_path, 'r', encoding='utf-8') as f:
        for line in f:
            w = line.strip()
            if w:
                stopwords.add(w)
    return stopwords

### 定义文本处理函数
这里定义两个函数用于判断 token 是否为全中文或全英文。

In [ ]:
def is_chinese(token):
    """判断 token 是否只包含中文字符"""
    return re.fullmatch(r'[\u4e00-\u9fa5]+', token) is not None

def is_english(token):
    """判断 token 是否只包含英文字母"""
    return re.fullmatch(r'[A-Za-z]+', token) is not None

### 配置参数
配置输入 JSONL 文件、输出 JSONL 文件、自定义词典和停用词表路径。

In [ ]:
input_jsonl = ""
output_jsonl = ""
dict_file = ""
stopword_file = ""

### 加载自定义词典
如果存在自定义词典，则加载，否则使用默认分词。

In [ ]:
if os.path.exists(dict_file):
    jieba.load_userdict(dict_file)
    print(f"已加载自定义词典: {dict_file}")
else:
    print(f"未找到自定义词典: {dict_file}, 将使用默认分词")

### 加载停用词
读取停用词文件，并加载到 `stopwords` 集合中。

In [ ]:
stopwords = load_stopwords(stopword_file)
print(f"已加载停用词，共 {len(stopwords)} 个")

### 读取 JSONL 并处理文本
读取 JSONL 文件，进行分词，并存储结果。

In [ ]:
data_list = []
with open(input_jsonl, 'r', encoding='utf-8') as fin:
    for line in fin:
        line = line.strip()
        if not line:
            continue
        record = json.loads(line)
        text = record.get("content", "")

        # 使用 jieba 分词
        tokens = jieba.lcut(text)
        tokens_filtered = []
        for token in tokens:
            # 去除零宽字符
            token = re.sub(r'[\u200b\u200c\u200d\ufeff]', '', token)
            token = token.strip()
            if not token or token in stopwords:
                continue
            # 只保留全中文或全英文的 token
            if is_chinese(token):
                if 2 <= len(token) <= 6:
                    tokens_filtered.append(token)
            elif is_english(token):
                if len(token) >= 3:
                    tokens_filtered.append(token)
            # 其他情况（数字、标点、表情或混合字符）均删除
        
        record["tokens"] = tokens_filtered
        data_list.append(record)

### 写入新的 JSONL 文件
将处理后的数据写入新的 JSONL 文件。

In [ ]:
with open(output_jsonl, 'w', encoding='utf-8') as fout:
    for rec in data_list:
        fout.write(json.dumps(rec, ensure_ascii=False) + "\n")

print(f"分词完成，共处理 {len(data_list)} 条记录，结果已写入 {output_jsonl}")